In [6]:
from myhdl import *

@block
def test(clk, a, b):
    @always(clk.posedge)
    def worker():
        b.next = a ^ 0xaa

    return instances()
        
@block
def tb():
    a, b = [ Signal(intbv()[8:]) for i in range(2) ]
    clk = Signal(bool())
    
    inst = test(clk, a, b)
    @instance
    def clkgen():
        while 1:
            clk.next = 0
            yield delay(10)
            clk.next = 1
            yield delay(10)
    
    @instance
    def verify():
        yield clk.posedge
        a.next = 0xaa
        for i in range(256):
            yield clk.posedge
            a.next = i
            yield clk.posedge
            print(b)

        raise StopSimulation()
    
    return instances()

    
t = tb()
t.config_sim()
t.run_sim(5000)
t.quit_sim()

00
aa
ab
a8
a9
ae
af
ac
ad
a2
a3
a0
a1
a6
a7
a4
a5
ba
bb
b8
b9
be
bf
bc
bd
b2
b3
b0
b1
b6
b7
b4
b5
8a
8b
88
89
8e
8f
8c
8d
82
83
80
81
86
87
84
85
9a
9b
98
99
9e
9f
9c
9d
92
93
90
91
96
97
94
95
ea
eb
e8
e9
ee
ef
ec
ed
e2
e3
e0
e1
e6
e7
e4
e5
fa
fb
f8
f9
fe
ff
fc
fd
f2
f3
f0
f1
f6
f7
f4
f5
ca
cb
c8
c9
ce
cf
cc
cd
c2
c3
c0
c1
c6
c7
c4
c5
da
db
d8
d9
de
df
dc
dd
d2
d3
d0


<class 'myhdl._SuspendSimulation'>: Simulated 5000 timesteps
